In [28]:
load('aca2.mat')
X = transpose(X);
s = transpose(s);

In [29]:
size(X)


ans =

        2204          42



In [30]:
size(s)


ans =

        2204           1



In [31]:
a = [1 2 3; 4 5 6];
a_row_norm = sqrt(sum(a.^2,2));
a_normalized = bsxfun(@rdivide, a, a_row_norm); 
sqrt(sum(a_normalized.^2,2))



ans =

    1.0000
    1.0000



In [32]:
X_truncated = X(1:2:end, :);
s_truncated = s(1:2:end);


In [38]:
X_col_norm = sqrt(sum(X_truncated.^2,1)); 
X_truncated = bsxfun(@rdivide,X_truncated, X_col_norm); 
X_col_norm = reshape(X_col_norm,[],1);
X_truncated(isnan(X_truncated)) = 0;


In [39]:
x = size(X_truncated,1 )


x =

        1102



In [40]:
gamma =1 ;
k = 2;

In [43]:
K = zeros(size(X_truncated, 1), size(X_truncated, 1));
for i=1:size(X_truncated,1)
    for j=1:size(X_truncated,1)
        K(i,j) = exp(-gamma* (norm( X_truncated(i, :) - X_truncated(j, :), 2).^2) );
    end
end     

In [44]:
K_maxk = maxk(K, k);
W = K;
for i=1:size(X_truncated,1)
    for j=1:size(X_truncated,1)
        if K(i,j) < min(K_maxk(:, j))
            W(i,j) = 0;
        end
    end
end

In [45]:
W = (W+transpose(W))/2;

In [48]:
D = diag(W*ones(size(W,1),1));

In [49]:
I = eye(size(W,1));

In [50]:
laplacian_unnormalized = D-W;
laplacian_normalized = I-W;
laplacian_rw = I-inv(D)*W;




In [51]:
[V1, D1] = eigs(laplacian_unnormalized, k);
[V2, D2] = eigs(laplacian_normalized, k);
[V3, D3] = eigs(laplacian_rw, k);


In [52]:
V1_row_norm = sqrt(sum(V1.^2,2));
Y1 = bsxfun(@rdivide, V1, V1_row_norm);

V2_row_norm = sqrt(sum(V2.^2,2));
Y2 = bsxfun(@rdivide, V2, V2_row_norm);

V3_row_norm = sqrt(sum(V3.^2,2));
Y3 = bsxfun(@rdivide, V3, V3_row_norm);


In [53]:
idx1 = kmeans(Y1, k);

In [54]:
idx2 = kmeans(Y2, k);
idx3 = kmeans(Y3, k);

In [56]:
size(idx1)


ans =

        1102           1



In [57]:
Misclassification([idx1 idx2 idx3], s_truncated)


ans =

    0.7740
    0.7541
    0.7713

